# Cleaning Data Exercises

<span style="color: #008080">*Bárbara Flores*</span>

In this exercise, we'll be returning to the American Community Survey data we used previously to measuring racial income inequality in the United States. In today's exercise, we'll be using it to measure the returns to education and how those returns vary by race and gender.



## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_missing.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}
```


### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

In [24]:
results = {}

## Exercises

### Exercise 1

For these cleaning exercises, we'll return to the ACS data we've used before one last time. We'll be working with `US_ACS_2017_10pct_sample.dta`. Import the data (please use url for the autograder).

In [25]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

data = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

data.sample(2)

,year,datanum,serial,cbserial,numprec,subsamp,hhwt,hhtype,cluster,adjust,...,migcounty1,migmet131,vetdisab,diffrem,diffphys,diffmob,diffcare,diffsens,diffeye,diffhear
64673,2017,1,1039440,2.017001e+12,2,25,270,married-couple family household,2.017010e+12,1.011189,...,0,not in identifiable area,n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no
310241,2017,1,992869,2.017001e+12,2,8,137,"male householder, no wife present",2.017010e+12,1.011189,...,153,"akron, oh",n/a,no cognitive difficulty,no ambulatory difficulty,no independent living difficulty,no,no vision or hearing difficulty,no,no


### Exercise 2

For our exercises today, we'll focus on `age`, `sex`, `educ` (education), and `inctot` (total income). Subset your data to those variables, and quickly look at a sample of 10 rows.

In [26]:
data_subset = data[["age", "sex", "educ", "inctot"]]
data_subset.sample(10)

,age,sex,educ,inctot
235744,34,male,4 years of college,487000
232267,40,female,5+ years of college,60000
122393,3,male,n/a or no schooling,9999999
3992,43,male,grade 12,25000
90034,15,male,"grade 5, 6, 7, or 8",0
221049,65,male,4 years of college,20000
111718,42,male,4 years of college,65000
65051,35,female,5+ years of college,38500
111578,59,female,4 years of college,15000
166965,46,female,1 year of college,15000


### Exercise 3

As before, all the values of `9999999` have the potential to cause us real problems, so replace all the values of `inctot` that are `9999999` with `np.nan`. 

In [27]:
data_subset["inctot"] = data_subset["inctot"].replace(9999999, np.nan)

### Exercise 4

Attempt to calculate the average age of people in our data. What do you get? Why are you getting that error?

You *should* get an error in trying to answer this question, but **PLEASE LEAVE THE CODE THAT GENERATES THIS ERROR COMMENTED OUT SO YOUR NOTEBOOK WILL RUN IN THE AUTOGRADER**. 

Then talk about the error in a markdown cell.

In [28]:
# data_subset["age"].mean()

In [29]:
data_subset["age"].dtype

CategoricalDtype(categories=['less than 1 year old', '1', '2', '3', '4', '5', '6', '7',
                  '8', '9', '10', '11', '12', '13', '14', '15', '16', '17',
                  '18', '19', '20', '21', '22', '23', '24', '25', '26', '27',
                  '28', '29', '30', '31', '32', '33', '34', '35', '36', '37',
                  '38', '39', '40', '41', '42', '43', '44', '45', '46', '47',
                  '48', '49', '50', '51', '52', '53', '54', '55', '56', '57',
                  '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
                  '68', '69', '70', '71', '72', '73', '74', '75', '76', '77',
                  '78', '79', '80', '81', '82', '83', '84', '85', '86', '87',
                  '88', '89', '90 (90+ in 1980 and 1990)', '91', '92', '93',
                  '94', '95', '96'],
, ordered=True, categories_dtype=object)

><span style="color: #008080">*When trying to directly calculate the average on the 'age' variable, I encountered the TypeError: 'Categorical' with dtype category does not support reduction 'mean.'*</span>
>
><span style="color: #008080">*If we observe the type of this variable, we notice that it is a categorical variable, which contains strings. We need to be careful when converting it to an integer since we have the categories 'less than 1 year old' and '(90+ in 1980 and 1990).*</span>
>
><span style="color: #008080">*One should consider some decision rules to calculate the mean. One option could be to eliminate these data points. Another option could be to replace 'less than 1 year old' with 0 and '(90+ in 1980 and 1990)' with 90. This decision depends on the distribution of the data and the researcher's criteria*</span>

### Exercise 5

We want to be able to calculate things using age, so we need it to be a numeric type. Check the current type of `age`, and look at all the values of `age` to figure out why it's categorical and not numeric. You should find two problematic categories. Store the values of these categories in `"ex5_age_young"` and `"ex5_age_old"` (once you find them, it should be clear which is which).

><span style="color: #008080">*As we saw in the previous exercise, our two problematic categories are **'less than 1 year old'** and **'(90+ in 1980 and 1990)**.*</span>

In [30]:
results["ex5_age_young"] = "less than 1 year old"
results["ex5_age_old"] = "90 (90+ in 1980 and 1990)"

### Exercise 6

In order to convert `age` into a numeric variable, we need to replace those problematic entries with values that `pandas` can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values. 

**Hint 1:** Categorical variables act like strings, so you might want to use string methods! 

**Hint 2:** Remember that characters like parentheses, pluses, asterisks, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

**Hint 3:** Because the US Census has been conducted regularly for hundreds of years but exactly how the census has been conducted have occasionally changed, variables are sometimes coded in a way that might be interpreted in different ways for different census years. For example, hypothetically, one might write `90 (90+ in 1980 and 1990)` if the Censuses conducted in 1980 and 1990 used to top-code age at 90 (any values *over* 90 were just coded as 90), but more recent Censuses no longer top-coded age and recorded ages over 90 as the respondents actual age.

In [31]:
data_subset["age"] = data_subset["age"].replace("less than 1 year old", "0")
data_subset["age"] = data_subset["age"].replace("90 (90+ in 1980 and 1990)", "90")

### Exercise 7

Now convert age from a categorical to numeric. Calculate the average age amoung this group, and store it in `"ex7_avg_age"`.

In [32]:
data_subset["age"] = data_subset["age"].astype(int)
ex7_avg_age = data_subset["age"].mean()
results["ex7_avg_age"] = ex7_avg_age

print(f"The average age among this sample from ACS is {round(ex7_avg_age,2)} years")

The average age among this sample from ACS is 41.3 years


### Exercise 8

Let's now filter out anyone in our data whose age is less than 18. Note that before made `age` a numeric variable, we couldn't do this! Again, calculate the average age and this time store it in `"ex8_avg_age"`. 

Use this sample of people 18 and over for all subsequent exercises.

In [33]:
data_subset = data_subset[data_subset["age"] >= 18]
ex8_avg_age = data_subset["age"].mean()
results["ex8_avg_age"] = ex8_avg_age
print(
    f"The average age among this sample from ACS, for individuals aged 18 or older, is {round(ex8_avg_age,2)} years"
)

The average age among this sample from ACS, for individuals aged 18 or older, is 49.76 years


### Exercise 9

Create an indicator variable for whether each person has *at least* a college Bachelor's degree called `college_degree`. Use this variable to calculate the number of people in the dataset with a college degree. You may assume that to get a college degree you need to complete at least 4 years of college. Save the result as `"ex9_num_college"`.

In [34]:
print("First, let's see how the 'educ' variable is distributed within our subset.")
data_subset["educ"].dtype

First, let's see how the 'educ' variable is distributed within our subset.


CategoricalDtype(categories=['n/a or no schooling', 'nursery school to grade 4',
                  'grade 5, 6, 7, or 8', 'grade 9', 'grade 10', 'grade 11',
                  'grade 12', '1 year of college', '2 years of college',
                  '4 years of college', '5+ years of college'],
, ordered=True, categories_dtype=object)

In [35]:
print(data_subset["educ"].value_counts().sort_index())

educ
n/a or no schooling           3644
nursery school to grade 4     1288
grade 5, 6, 7, or 8           5975
grade 9                       3145
grade 10                      4078
grade 11                      5816
grade 12                     92576
1 year of college            38746
2 years of college           20753
4 years of college           47212
5+ years of college          29801
Name: count, dtype: int64


><span style="color: #008080">*"When observing the 'educ' variable, we can see that it is a categorical variable, and its categories are already ordered. Therefore, we can use comparison operators such as >=, <=, <, > without any issues."*</span>

In [36]:
data_subset["college_degree"] = data_subset["educ"] >= "4 years of college"
data_subset

,age,sex,educ,inctot,college_degree
2,63,male,4 years of college,6150.0,True
3,66,female,grade 12,14000.0,False
5,50,male,grade 12,50000.0,False
6,82,male,1 year of college,27100.0,False
10,47,male,n/a or no schooling,18000.0,False
...,...,...,...,...,...
318998,31,female,5+ years of college,0.0,True
318999,33,female,4 years of college,22130.0,True
319001,20,male,grade 12,5000.0,False
319002,47,male,5+ years of college,240000.0,True


In [37]:
ex9_num_college = data_subset[data_subset["college_degree"] == True].shape[0]
results["ex9_num_college"] = ex9_num_college
print(
    f"The number of people with a college degree among this sample from ACS is {round(ex9_num_college, 2):,} people."
)

The number of people with a college degree among this sample from ACS is 77,013 people.


### Exercise 10

Let's examine how the educational gender gap. Use `pd.crosstab` to create a cross-tabulation of `sex` and `college_degree`. `pd.crosstab` will give you the number of people who have each combination of `sex` and `college_degree` (so in this case, it will give us a 2x2 table with Male and Female as rows, and `college_degree` True and False as columns, or vice versa. 

In [38]:
pd.crosstab(data_subset["sex"], data_subset["college_degree"])

college_degree,False,True
sex,,
male,85821,36181
female,90200,40832


### Exercise 11

Counts are kind of hard to interpret. `pd.crosstab` can also normalize values to give percentages. Look at the `pd.crosstab` help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees.

Store the share (between 0 and 1) of men with college degrees in `"ex11_share_male_w_degrees"`, and the share of women with degrees in `"ex11_share_female_w_degrees"`.

In [39]:
table = pd.crosstab(
    data_subset["sex"], data_subset["college_degree"], normalize="index"
)
print(table)

college_degree     False     True 
sex                               
male            0.703439  0.296561
female          0.688381  0.311619


In [40]:
ex11_share_male_w_degrees = table[True]["male"]
ex11_share_female_w_degrees = table[True]["female"]

results["ex11_share_male_w_degrees"] = ex11_share_male_w_degrees
results["ex11_share_female_w_degrees"] = ex11_share_female_w_degrees

print(
    f"The percentage of men aged 18 or older with a college degree among this sample from ACS is {round(ex11_share_male_w_degrees*100, 2):,}%."
)
print(
    f"The percentage of female aged 18 or older with a college degree among this sample from ACS is {round(ex11_share_female_w_degrees*100, 2):,}%."
)

The percentage of men aged 18 or older with a college degree among this sample from ACS is 29.66%.
The percentage of female aged 18 or older with a college degree among this sample from ACS is 31.16%.


><span style="color: #008080">*We can see that the percentage of men with a college degree in this sample is similar to that of women, with the women's percentage being slightly higher*</span>

### Exercise 12

Now, let's recreate that table for people who are 40 and over and people under 40. Over time, what does this suggest about the absolute difference in the share of men and women earning college degrees? Has it gotten larger, stayed the same, or gotten smaller? Store your answer (either `"the absolute difference has increased"` or `"the absolute difference has decreased"`) in `"ex12_comparing"`.

In [41]:
data_subset["40_90"] = data_subset["age"] >= 40
data_subset

,age,sex,educ,inctot,college_degree,40_90
2,63,male,4 years of college,6150.0,True,True
3,66,female,grade 12,14000.0,False,True
5,50,male,grade 12,50000.0,False,True
6,82,male,1 year of college,27100.0,False,True
10,47,male,n/a or no schooling,18000.0,False,True
...,...,...,...,...,...,...
318998,31,female,5+ years of college,0.0,True,False
318999,33,female,4 years of college,22130.0,True,False
319001,20,male,grade 12,5000.0,False,False
319002,47,male,5+ years of college,240000.0,True,True


In [42]:
data_subset_18_39 = data_subset[data_subset["40_90"] == False]
data_subset_40_90 = data_subset[data_subset["40_90"] == True]

In [43]:
table2 = pd.crosstab(
    data_subset_18_39["sex"], data_subset_18_39["college_degree"], normalize="index"
)

table3 = pd.crosstab(
    data_subset_40_90["sex"], data_subset_40_90["college_degree"], normalize="index"
)
print("18 to 39 years: \n")
print(table2)
print("\n40+ years: \n")
print(table3)

18 to 39 years: 

college_degree     False     True 
sex                               
male            0.743143  0.256857
female          0.665710  0.334290

40+ years: 

college_degree     False     True 
sex                               
male            0.682123  0.317877
female          0.699144  0.300856


In [44]:
results["ex12_comparing"] = "the absolute difference has increased"

print(
    f"The table shows the absolute difference of male and female who are over 40 and have college degress has increased in this sample."
)

The table shows the absolute difference of male and female who are over 40 and have college degress has increased in this sample.


### Exercise 13

In words, what is causing the change noted in Exercise 12 (i.e., looking at the tables above, tell me a story about Men and Women's College attainment).

><span style="color: #008080">*From the analyzed sample, we can draw the following conclusions:*</span>
>
><span style="color: #008080">*- In newer generations, a higher percentage of individuals are completing four-year college degrees. This trend is observed for both women and men.*</span>
>
><span style="color: #008080">*- We can observe an evolution in terms of gender equity in college education. Previously, the proportion of men pursuing four-year college education was higher than that of women, but this gap has narrowed in recent times.*</span>

## Check results

In [45]:
results

{'ex5_age_young': 'less than 1 year old',
 'ex5_age_old': '90 (90+ in 1980 and 1990)',
 'ex7_avg_age': 41.30384885455982,
 'ex8_avg_age': 49.75769659413359,
 'ex9_num_college': 77013,
 'ex11_share_male_w_degrees': 0.29656071211947344,
 'ex11_share_female_w_degrees': 0.3116185359301545,
 'ex12_comparing': 'the absolute difference has increased'}

In [46]:
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}

## Want More Practice?

Calculate the educational racial gap in the United States for White Americans, Black Americans, Hispanic Americans, and other groups. 

Note that to do these calculations, you'll have to deal with the fact that unlike most Americans, the American Census Bureau treats "Hispanic" not as a racial category, but a linguistic one. As a result, the racial category "White" in `race` actually includes most Hispanic Americans. For this analysis, we wish to work with the mutually exclusive categories of "White, non-Hispanic", "White, Hispanic", "Black (Hispanic or non-Hispanic)", and a category for everyone else. 